In [ ]:
#======================================
# ORB Code Structure
#======================================

# === Imports ===
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from pathlib import Path

# === Settings ===
input_folder = "Input File Path"  # <----- Update file path
output_folder = "Output File Path" # <----- Update file path
Path(output_folder).mkdir(exist_ok=True)

align_images = True
apply_histogram_matching = False  # Set to True if lighting changes are strong
gaussian_blur_kernel = (5, 5)

# === Functions ===
def align_image(ref_img, target_img):
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(ref_img, None)
    kp2, des2 = orb.detectAndCompute(target_img, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    M, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts)
    aligned = cv2.warpAffine(target_img, M, (ref_img.shape[1], ref_img.shape[0]))
    return aligned

def plot_heatmap(diff_img, img_name, output_path):
    plt.figure(figsize=(10, 6))
    im = plt.imshow(diff_img, cmap='seismic', vmin=-256, vmax=256)
    plt.colorbar(im, label="Pixel Intensity Difference")
    plt.title(f"Aligned Image Difference Heatmap (Red = darker, Blue = lighter)\n{img_name}")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()

# === Main Loop ===
image_paths = sorted(glob(os.path.join(input_folder, "*.bmp"))) #<----- UPDATE IMAGE TYPE - WILL NOT WORK UNLESS IT MATCHES
assert len(image_paths) >= 2, "You need at least two images to compare!"

ref_img_raw = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
ref_img = cv2.GaussianBlur(ref_img_raw, gaussian_blur_kernel, 0)

for i in range(1, len(image_paths)):
    img_path = image_paths[i]
    img_name = f"{Path(img_path).stem} - {Path(image_paths[0]).stem}"
    
    current_img_raw = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Optional: histogram match to reference (if needed)
    if apply_histogram_matching:
        from skimage.exposure import match_histograms
        current_img_raw = match_histograms(current_img_raw, ref_img_raw, channel_axis=None).astype(np.uint8)
    
    current_img_blur = cv2.GaussianBlur(current_img_raw, gaussian_blur_kernel, 0)
    aligned_img = align_image(ref_img, current_img_blur) if align_images else current_img_blur

    # === Image Subtraction  ===
    diff_img = aligned_img.astype(np.float32) - ref_img.astype(np.float32)

    heatmap_path = f"{output_folder}/heatmap_{img_name}.png"

    # Save heatmap with proper centering
    plot_heatmap(diff_img, img_name, heatmap_path)

    print(f"Processed {img_name}")

print("ORB alignment and subtraction complete!")
